Examine reading out different address,bit combinations

In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
import pandas as pd
from tqdm.notebook import trange, tqdm

soc = TopSoc()

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
fMixer = soc.fsIn/2/2 + 0.25
scan.soc.set_mixer(fMixer)

In [ ]:
# Setup tones that are in nearby channels, to fully test multi-tone readout
def addrBit2Inchs(addr, bit):
    ntran = 32*addr + bit
    chs = ntran*8 + np.arange(8)
    return chs

version = 0

if version==0:
    f0 = fMixer + 120.4
    nTones = 10
    freqs = f0 + 0.1 + np.arange(nTones)*(1.2*soc.fcOut+0.01)
elif version==1: # works, except for decrementing one tone
    f0 = fMixer + 120.4
    nTones = 10
    freqs = f0 + 0.1 + np.arange(nTones)*(1.2*soc.fcOut+0.01)
    # Move one tone away from the others
    freqs[1] += 334.56 # ok to have this line
    freqs[3] -= 134.56 # uncommenting this line causes trouble -= but works with +=
    freqs[4] += 243.2 # ok to have this line
elif version==3:
    # Define tones by addr,bit combination.  Choose only 2 of them to avoid collisions
    addr = 2
    bit = 3
    freqs = 0.01 + soc.inCh2FreqCenter(addrBit2Inchs(addr,bit))[1::4] # 
elif version==4:
    addr = 1
    bit = 3
    freqs = 0.01 + soc.inCh2FreqCenter(addrBit2Inchs(addr,bit))[::4] # 
    freqs = np.append(freqs, 0.01 + soc.inCh2FreqCenter(addrBit2Inchs(addr+1,bit+1))[2::4]) # 
    freqs = np.append(freqs, 0.01 + soc.inCh2FreqCenter(addrBit2Inchs(addr+2,bit+2))[1::4]) # 
else:
    raise ValueError()
print(freqs)
type(freqs)

In [ ]:
def freq2NtransAddrBit(self, freq):
    inChs = self.inFreq2ch(freq)
    ntrans, addrs, bits = self.chsel.ch2tran(inChs) # bit ranges from 0 to 31 
    #addrs = (inChs / (8*32)).astype(int)
    #addrs,ks = np.divmod(inChs, 8*32, dtype=int)
    return ntrans, addrs, bits

def ntranAddrBit2Ch(self, ntran, addr, bit):
    ch = ntran*self.chsel.L
    print("calculated ch =",ch)
    


In [ ]:
freqs = np.sort(freqs)
pfbInCh, dds_freq, pfb_freq, ch = soc.pfb_in.freq2ch(freqs)   
outCh, outOffset = soc.outFreq2chOffset(freqs)
inCh, inOffset = soc.inFreq2chOffset(freqs)
ntrans, addrs, bits = freq2NtransAddrBit(soc, freqs)
data = np.transpose(np.array([freqs, outCh, outOffset, inCh, inOffset, pfbInCh, dds_freq, pfb_freq,ch, ntrans, addrs, bits]))
pd.DataFrame(data, columns=["freq","outCh","outOffset", "inCh", "inOffset","pfbInCh", "dds_freq","pfb_freq","ch", "ntrans", "addrs","bits"])

In [ ]:
chatty=False
amps = 0.9*np.ones(len(freqs))/len(freqs)
fis = 0.5+(0.2*np.arange(len(freqs)))
scan.setTones(freqs, amps, fis)
decimation = 2
scan.prepRead(decimation, verbose=chatty, debugChselSet=True)

nt = 1
nsamp = 20000
truncate = 1000
x = scan.readAndUnpack(nt=nt, nsamp=nsamp,unpackVerbose=False, subtractInputPhase=False)
packets = scan.p.packets
input_config = scan.p.input_config
print("  number of good samples =",scan.p.nGoodSamp)
print("     i16Pattern expected =",scan.p.input_config['i16Pattern'])
print("i16Pattern ending actual =",end=" [")
for i in range(-2*len(scan.p.input_config['i16Pattern']), 0):
    print(scan.p.i16s[i],end=", ")
print("]")
for itone in range(len(freqs)):
    print("mean amplitude of freq=%f MHz tone  %2d = %f"%(freqs[itone],itone,np.abs(x[itone,:]).mean()))
#print("mean amplitude of tone -1 =",np.abs(x[-1,:]).mean())

In [ ]:
for itone in range(len(freqs)):
    plt.plot(np.angle(x[itone,:]),label=itone)
plt.legend()
plt.ylabel("phase [Radians]")
plt.xlabel('sample number')